In [ ]:
#Import Packages
#If You dont have some of the below packages you have to install first.  pip install "package" 
from selenium import webdriver
import time
import pandas as pd
import os
import requests


In [ ]:

#Import Packages
#If You dont have some of the below packages you have to install first.  pip install "package" 
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup


In [ ]:
#Variables to change in this part. We going to use the same path to chromeDriver and the list of tecnology waht we want to search in the body of propose jobs
#path to CrhromeDriver
pathToChromeDriver=r'C:\Users\patricio\Downloads\chromedriver_win32\chromedriver.exe'
#Tecnology and requirements list. If any of this words can be find in a compound word we try to search the word alone like this: ' SQL ','>SQL ',' SQL<'.        
Tecnologies=['Power BI','Visme','Google Charts','Google Analytics','Ingles','Excel','Python',' SQL ','>SQL ',' SQL<','Big Data','MongoDB','NoSQL','PostgreSQL','MariaDB','Looker','AWS',
             'English','Tableau','Qlik'," R ",' R<',">R ",'Julia','Microsoft Office','Pandas','Seaborn','Linux','Scrum','Jira','GitHub','Gitlab','Docker','Kubernetes','metodologías ágiles',
             'Matplotlib','Machine Learning','Deep Learning','Keras','TensorFlow','Scikit learn','Plotly','NumPy','Bokeh','SciPy','Artificial Inteligence','inteligencia artificial',
            'Kanban','Extreme Programming','Scala','business intelligence','business analyst', 'Access','MySQL',' SAP ','>SAP ',' SAP<','Power Point','Word']


In [ ]:
#In this part we save the link
service = Service(executable_path=pathToChromeDriver)
#We use the same csv which created in the first part.
#path to csv:
DataFrameOfJobs=pd.read_csv(r'linkedinDate1002.csv')
urllink=DataFrameOfJobs['link']

DataFrameOfJobs

Database Extraction

In [ ]:
#Principal part of the code, in this cell we extract the list technology and the count of every word in the list. this part going to every link
#of the proposes jobs and will search the technology words in the technology list.
finalListForDataframe= []
for i in range(len(urllink)):
    print(urllink[i].index)
    Test = False
    TecnologiesForDataFrame = {}
    r = requests.get(urllink[i])
    time.sleep(2)
    soup = BeautifulSoup(r.text, 'lxml')
    for Countator in Tecnologies:
        try : 
            TecnologiesForDataFrame[Countator]= (str(soup.find(class_='show-more-less-html__button').parent.parent.section).lower()).count(Countator.lower())
        except AttributeError:
            DataFrameOfJobs=DataFrameOfJobs.drop(i)
            Test = True
            print('Different format or the propose dont acept solicitude anymore')
            
            break
            
    if Test == False:
        finalListForDataframe.append(TecnologiesForDataFrame)  
    
#This part will be the slower part of the code. "I add one script where we see if the code still running"

Creacion del DataFramePrincipal.

In [ ]:
#Index of DataframeJobs
index = []
for i in range(len(DataFrameOfJobs)):
    index.append(i)
DataFrameOfJobs['index']= index
DataFrameOfJobs.set_index('index')

In [ ]:
df=pd.DataFrame(finalListForDataframe)
DataFrameOfJobs=DataFrameOfJobs.set_index('index')


 ##### Clean data and appends to the tables####

In [ ]:
df['links']= DataFrameOfJobs['link']
df['Title']= DataFrameOfJobs['title']
df['Company']= DataFrameOfJobs['company']
df['Place']= DataFrameOfJobs['JobPlace']
df['Date']= DataFrameOfJobs['dateJob']

df['R']= df[' R ']+df['>R ']+df[' R<']
df=df.drop(columns=[' R ','>R ',' R<'])
df['SQL']= df[' SQL ']+df['>SQL ']+df[' SQL<']
df['SAP']= df[' SAP ']+df['>SAP ']+df[' SAP<']
df=df.drop(columns=['>SQL ',' SQL<'])
df=df.drop(columns=[' SAP ','>SAP ',' SAP<'])


In [ ]:
df ['test']=df.sum(axis=1)
list(df['test']).count(0)


Tranformation and saved in the local.


In [ ]:
#save in local the Main DataFrame
df.to_csv('linkedinDate1002.csv')

show the generated table.

In [ ]:
freq = df.groupby(['Company']).count().sort_values(by=['Title'],ascending=False                                               )
#this code print the company with more job propose


Presents the number of jobs generated by company

In [ ]:
freq['Title']

Transformo date a fecha.




In [ ]:
import datetime
pd.options.display.max_rows = 180

df=pd.read_csv('linkedinDate1002.csv')
df=df.drop(columns='Unnamed: 0')


In [ ]:
#In  this part i change format of the date saved in the  Dataframe created previously. In this way we could use the new date as a datetime column
pd.options.mode.chained_assignment = 'warn'
actual_date = datetime.datetime.now()
dates = []

for i in range(len(df)):
    datesContator = False
    print(i)
    time.sleep(0.01)
    for week in range(5):
        if f'{week} semana' in df.iloc[i]['Date']:
            dates.append(actual_date - datetime.timedelta(weeks=week))
            datesContator = True
    for day in range(8):
        if f'Hace {day} d' in df.iloc[i]['Date']:
            dates.append(actual_date - datetime.timedelta(days=day))
            datesContator = True
    for month in range(13):
        if f'{month} mes' in df.iloc[i]['Date']:
            dates.append(actual_date - datetime.timedelta(days=30*month))
            datesContator = True

    for minutes in range(60):
        if f'{minutes} minute' in df.iloc[i]['Date']:
            dates.append(actual_date - datetime.timedelta(minutes=minutes))
            datesContator = True
    if datesContator == False:
            dates.append(actual_date - datetime.timedelta(weeks=1))
            print(i)
            print(df.iloc[i]['Date'])
print(f'{len(dates)},  {len(df)}')
df['DatesTime']=dates

#In the next part we created another csv where we going to save a dataframe with every technology if the program find more than word same in the "job propose" only save 1

In [ ]:
df=pd.read_csv('linkedinDate1002.csv')

In [ ]:
pd.options.mode.chained_assignment = None
UniqueTecnology=df

for column in UniqueTecnology.columns:
    if column in ['Unnamed: 0','links','Title','Company','Place','Date','DatesTime','test']:
        continue
    for rows in range(len(UniqueTecnology)): 
        try:
            if UniqueTecnology.iloc[rows][column] >=1:
                UniqueTecnology.loc[rows, column] = 1
        except:
            print(UniqueTecnology.iloc[rows][column])

In [ ]:
#this two csv are the two main csv for the data analyst , the first only count 1 for every tech requirement, and the second count the number of tomes the same technology requirement is repeated in the same offer.
UniqueTecnology.to_csv('TabUniqueTecnology.csv')
df.to_csv('TableauFinal.csv') 